# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /home/brad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/brad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/brad/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [13]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [14]:
# load data from database
engine = create_engine('sqlite:///DisasterDatabase.db')
df = pd.read_sql_table(con=engine, table_name='Disaster')
X = df['message']
Y = df.iloc[:,4:] 

In [15]:
engine

Engine(sqlite:///DisasterDatabase.db)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [37]:
# PRELIMINARY PIPELINE
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=56)),
    ])

    return pipeline

# NEW PIPELINE (AFTER SOME TRIAL-AND-ERROR ALGORITHM TEST AND GRID SEARCH)
def new_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, learning_rate=0.5),n_jobs=56))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = new_model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
y_pred = model.predict(X_test)

In [21]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [22]:
y_test = y_test.reset_index(drop=True)
y_pred = pd.DataFrame(y_pred)

In [23]:
labels = y_test.columns

In [24]:
for x in range(len(y_test.columns)):
    print(labels[x])
    print(classification_report(y_test.iloc[:,x], y_pred.iloc[:,x]))
    print('\n')
    

related
              precision    recall  f1-score   support

           0       0.56      0.24      0.34      1519
           1       0.80      0.94      0.86      4994
           2       0.20      0.12      0.15        41

    accuracy                           0.77      6554
   macro avg       0.52      0.43      0.45      6554
weighted avg       0.74      0.77      0.74      6554



request
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      5440
           1       0.74      0.54      0.63      1114

    accuracy                           0.89      6554
   macro avg       0.83      0.75      0.78      6554
weighted avg       0.88      0.89      0.88      6554



offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50     

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6413
           1       0.59      0.26      0.36       141

    accuracy                           0.98      6554
   macro avg       0.79      0.63      0.68      6554
weighted avg       0.98      0.98      0.98      6554



other_weather
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6226
           1       0.51      0.16      0.24       328

    accuracy                           0.95      6554
   macro avg       0.73      0.58      0.61      6554
weighted avg       0.93      0.95      0.94      6554



direct_report
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      5313
           1       0.71      0.48      0.57      1241

    accuracy                           0.86      6554
   macro avg       0.80      0.72      0.75      6554
weighted avg       0.85      0.86      0.8

### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
parameters = {'clf__estimator__base_estimator': [DecisionTreeClassifier(max_depth=1),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=5)],
             'clf__estimator__n_estimators': [50, 100],
             'clf__estimator__learning_rate':[1.,.75,.5]}

cv = GridSearchCV(model, param_grid=parameters)

In [26]:
grid = cv.fit(X_train, y_train)

/home/brad/.conda/envs/fastai/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
grid.best_score_
grid.best_params_

{'clf__estimator__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'),
 'clf__estimator__learning_rate': 0.5,
 'clf__estimator__n_estimators': 100}

In [28]:
best_model=grid.best_estimator_.fit(X_train, y_train)

In [29]:
best_y_pred = pd.DataFrame(best_model.predict(X_test))

for x in range(len(y_test.columns)):
    print(labels[x])
    print(classification_report(y_test.iloc[:,x], best_y_pred.iloc[:,x]))
    print('\n')

related
              precision    recall  f1-score   support

           0       0.62      0.20      0.30      1519
           1       0.79      0.96      0.87      4994
           2       0.40      0.10      0.16        41

    accuracy                           0.78      6554
   macro avg       0.60      0.42      0.44      6554
weighted avg       0.75      0.78      0.73      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5440
           1       0.79      0.52      0.62      1114

    accuracy                           0.89      6554
   macro avg       0.85      0.74      0.78      6554
weighted avg       0.89      0.89      0.89      6554



offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50     

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5934
           1       0.90      0.75      0.82       620

    accuracy                           0.97      6554
   macro avg       0.94      0.87      0.90      6554
weighted avg       0.97      0.97      0.97      6554



cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6413
           1       0.71      0.26      0.37       141

    accuracy                           0.98      6554
   macro avg       0.84      0.63      0.68      6554
weighted avg       0.98      0.98      0.98      6554



other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6226
           1       0.48      0.08      0.14       328

    accuracy                           0.95      6554
   macro avg       0.72      0.54      0.56      6554
weighted avg       0.93      0.95      0.93      65

/home/brad/.conda/envs/fastai/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
def new_model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=56)),
    ])

    return pipeline

In [37]:
new_model = new_model_pipeline()
new_model.fit(X_train, y_train)
new_y_pred = pd.DataFrame(model.predict(X_test))

for x in range(len(y_test.columns)):
    print(labels[x])
    print(classification_report(y_test.iloc[:,x], new_y_pred.iloc[:,x]))
    print('\n')

related
              precision    recall  f1-score   support

           0       0.56      0.30      0.39      1536
           1       0.81      0.93      0.86      4970
           2       0.15      0.06      0.09        48

    accuracy                           0.77      6554
   macro avg       0.51      0.43      0.45      6554
weighted avg       0.74      0.77      0.75      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5373
           1       0.79      0.54      0.64      1181

    accuracy                           0.89      6554
   macro avg       0.85      0.76      0.79      6554
weighted avg       0.89      0.89      0.88      6554



offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.11      0.03      0.05        29

    accuracy                           0.99      6554
   macro avg       0.55      0.52      0.52     

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6399
           1       0.66      0.32      0.43       155

    accuracy                           0.98      6554
   macro avg       0.82      0.66      0.71      6554
weighted avg       0.98      0.98      0.98      6554



other_weather
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6198
           1       0.47      0.14      0.22       356

    accuracy                           0.94      6554
   macro avg       0.71      0.57      0.60      6554
weighted avg       0.93      0.94      0.93      6554



direct_report
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      5231
           1       0.73      0.51      0.60      1323

    accuracy                           0.86      6554
   macro avg       0.81      0.73      0.76      6554
weighted avg       0.85      0.86      0.8

### 9. Export your model as a pickle file

In [30]:
# From best model grid search

from sklearn.externals import joblib
# Output a pickle file for the model
joblib.dump(best_model, 'saved_model.pkl') 

/home/brad/.conda/envs/fastai/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['saved_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [39]:
with open('saved_model.pkl', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)